In [1]:
!pip install langchain-google-genai

# Trying Langchain with Gemini

> Without any props in the Google provider

> Importing the OS to provide the environment variables

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

> Inserting the Gemini api directly

In [3]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [35]:
model_name = "gemini-pro"
llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)

> trying to parse pdf directly

In [4]:
!pip install pypdf
!pip install langchain_community

In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
import json

In [33]:
loader = PyPDFLoader("docs/test.pdf")
pages = loader.load_and_split()

template = """
Given this description: 
{content}
I want to know the name and the github of the user.
"""

qs = template.format(content=pages[0].page_content)

chain = llm

chain.invoke(qs).content

'Name: Mohamed Dhia Kassab\nGithub: github.com/meddhiaka'

In [30]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class JsonTemplate(BaseModel):
    name: str = Field(description = "name of the person")
    github_link: str = Field(description = "github of the person")

In [31]:
parser = JsonOutputParser(pydantic_object=JsonTemplate)

template = """
Please provide information about a person in JSON format.
{format_instructions}
Query: {query}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


chain = prompt | llm | parser

str_dict = chain.invoke({"query": qs})

json.dumps(str_dict, indent=4)


In [34]:
with open('data.json', 'w') as f:
    json.dump(str_dict, f)

str_dict

{'name': 'Mohamed Dhia Kassab', 'github_link': 'github.com/meddhiaka'}